In [19]:
from torch.utils.data import DataLoader, Dataset
import torch
import argparse
import numpy as np
import glob
from sklearn.manifold import TSNE
from tqdm import tqdm
import random
import os
import sys
import torch.optim as optim
import torch.utils.data
import numpy as np
import torch.nn as nn
from torchvision import transforms
from torch.autograd import Function
from PIL import Image
import os
import pandas as pd
import argparse

In [20]:
# class ReverseLayerF(Function):

#     @staticmethod
#     def forward(ctx, x, alpha):
#         ctx.alpha = alpha

#         return x.view_as(x)

#     @staticmethod
#     def backward(ctx, grad_output):
#         output = grad_output.neg() * ctx.alpha

#         return output, None



class DANN(nn.Module):
    def __init__(self, code_size=512, n_class=10):
        super(DANN, self).__init__()
        
        self.feature_extractor_conv = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=5),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(2),
            nn.ReLU(True),
            nn.Conv2d(64, 50, kernel_size=5),
            nn.BatchNorm2d(50),
            nn.Dropout2d(),
            nn.MaxPool2d(2),
            nn.ReLU(True)
        )

        self.feature_extractor_fc = nn.Sequential(
            nn.Linear(50 * 4 * 4, code_size),
            nn.BatchNorm1d(code_size),
            nn.Dropout(),
            nn.ReLU(True)
        )
        
        self.class_classifier = nn.Sequential(
            nn.Linear(code_size, 100),
            nn.BatchNorm1d(100),
            nn.ReLU(True),
            nn.Linear(100, n_class),
            nn.LogSoftmax(dim=1)
        )

        self.domain_classifier = nn.Sequential(
            nn.Linear(code_size, 100),
            nn.BatchNorm1d(100),
            nn.ReLU(True),
            nn.Linear(100, 2),
            nn.LogSoftmax(dim=1)
        )

    def forward(self, x):
        feature = self.feature_extractor_conv(x)
        feature = feature.view(-1, 50 * 4 * 4)
        feature = self.feature_extractor_fc(feature)

        return feature
    # def forward(self, x, alpha=1.0):
    #     feature = self.feature_extractor_conv(x)
    #     feature = feature.view(-1, 50 * 4 * 4)
    #     feature = self.feature_extractor_fc(feature)
    #     reverse_feature = ReverseLayerF.apply(feature, alpha)
    #     class_output = self.class_classifier(feature)
    #     domain_output = self.domain_classifier(reverse_feature)
        
    #     return class_output, domain_output
class TrainSet(Dataset):
    def __init__(self, name, mode, transform):
        self.path = os.path.join(data_path, name, mode)
        self.data = sorted(os.listdir(self.path))
        df = pd.read_csv(os.path.join(data_path, name, mode + '.csv'))
        self.label = dict(zip(df.image_name, df.label))
        self.transform = transform
        
    def __getitem__(self, index):
        return self.transform(Image.open(os.path.join(self.path, self.data[index])).convert('RGB')), int(self.label[self.data[index]])

    def __len__(self):
        return len(self.data)


In [21]:
class myDataset(Dataset):
    def __init__(self, root, label_data=None, transform=None):
        self.transform = transform
        self.label_data = label_data

        self.img_paths = []
        self.labels = []

        if label_data is not None:
            for d in self.label_data:
                img_path, label = d.split(',')
                self.img_paths.append(os.path.join(root, img_path))
                self.labels.append(int(label))
        else:
            for fn in glob.glob(os.path.join(root, '*.png')):
                self.img_paths.append(fn)
                self.labels.append(0)

        self.len = len(self.img_paths)


    def __getitem__(self, index):
        """ Get a sample from the dataset """
        image_fn, label = self.img_paths[index], self.labels[index]
        image = Image.open(image_fn).convert('RGB')
            
        if self.transform is not None:
            image = self.transform(image)

        return image, label

    def __len__(self):
        """ Total number of samples in the dataset """
        return self.len

In [22]:
batch_size = 32
image_size = 28
# transform = transforms.Compose([
#         transforms.ToTensor(),
#         transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
# ])
target_tfm = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize((.5, .5, .5), (.5, .5, .5))
])


data_path = '../hw2_data/digits'
tgt_domain = 'mnistm'
target_dataset = TrainSet(tgt_domain, 'test', target_tfm)
target_loader = DataLoader(target_dataset, batch_size = batch_size, shuffle = True, num_workers = 2)
# tgt_root = os.path.join(data_path, tgt_domain, 'train')
# tgt_trainset = myDataset(root=tgt_root, transform=transform)
# target_loader = DataLoader(tgt_trainset, batch_size=batch_size, shuffle=True, num_workers=1)

In [23]:

model = DANN().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3,  betas=[0.9, 0.999])
state = torch.load('./mnistm-mnistm/best.pth')
model.load_state_dict(state['state_dict'])
optimizer.load_state_dict(state['optimizer'])
# model.state_dict(torch.load('./mnistm-mnistm/best.pth'))
# optimizer.state_dict(torch.load('./mnistm-mnistm/best.pth'))

In [26]:
features, labels = [], []
for batch in tqdm(target_loader):
    imgs, lbls = batch
    with torch.no_grad():
        logits = model(imgs.cuda())
    labels += ['mnistm'] * len(logits)
    logits = np.squeeze(logits)
    features.extend(logits)
print(len(features), len(labels))

100%|██████████| 313/313 [00:01<00:00, 210.28it/s]

10000 10000


In [27]:
from sklearn.manifold import TSNE
colors_per_class = [list(np.random.choice(range(256), size=3)) for i in range(2)]
tsne = TSNE(n_components=2, init = 'pca').fit_transform(features)

ValueError: only one element tensors can be converted to Python scalars

In [28]:
import matplotlib.pyplot as plt

def scale_to_01_range(x):
    value_range = (np.max(x) - np.min(x))
    starts_from_zero = x - np.min(x)

    return starts_from_zero / value_range

def visualize_tsne_points(tx, ty, labels):
    fig = plt.figure()
    ax = fig.add_subplot(111)

    for idx, label in ['svhn', 'mnistm']:
        indices = [i for i, l in enumerate(labels) if l == label]

        current_tx = np.take(tx, indices)
        current_ty = np.take(ty, indices)

        color = np.array([colors_per_class[label][::-1]], dtype=np.float) / 255

        ax.scatter(current_tx, current_ty, c=color, label=label, s=5)
    ax.legend(loc='best')

    plt.show()

In [29]:
def visualize_tsne(tsne, labels, plot_size=1000, max_image_size=100):
    tx = tsne[:, 0]
    ty = tsne[:, 1]
    tx = scale_to_01_range(tx)
    ty = scale_to_01_range(ty)
    visualize_tsne_points(tx, ty, labels)

In [30]:
from pylab import rcParams
rcParams['figure.figsize'] = 10, 10

In [31]:
colors_per_class = [list(np.random.choice(range(256), size=3)) for i in range(50)]
visualize_tsne(tsne, labels)

NameError: name 'tsne' is not defined